In [ ]:
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"Successfully installed {package}")
    except Exception as e:
        print(f"Error installing {package}: {e}")

# Install compatible versions
packages_to_install = [
    "openai>=1.0.0",
    "crewai>=0.28.0",
    "crewai-tools>=0.4.0",
    "langchain-openai>=0.1.0",
    "pydantic>=2.0.0"
]

for package in packages_to_install:
    install_package(package)

In [ ]:
#Warning controls
import warnings
warnings.filterwarnings('ignore')

In [3]:
from langchain_openai import ChatOpenAI
import os

# Set environment variables
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""

# Disable RAG storage to avoid APIStatusError issues
os.environ["CREWAI_STORAGE_DIR"] = ""
os.environ["DISABLE_CREWAI_MEMORY"] = "true"

In [4]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import PyPDF2   
import docx    
import json
from typing import Dict, List
import os

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
from crewai_tools import FileReadTool, SerperDevTool
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

file_read_tool = FileReadTool()
serper_dev_tool = SerperDevTool()

# Custom Sentiment Analysis Tool using new CrewAI syntax
class SentimentAnalysisInput(BaseModel):
    """Input schema for SentimentAnalysisTool."""
    text: str = Field(..., description="Text to analyze for sentiment")

class SentimentAnalysisTool(BaseTool):
    name: str = "SentimentAnalysisTool"
    description: str = "Analyzes the sentiment of text to ensure positive and engaging communication."
    args_schema: Type[BaseModel] = SentimentAnalysisInput

    def _run(self, text: str) -> str:
        return "positive"

sentiment_analysis_tool = SentimentAnalysisTool()

In [6]:
from crewai import Agent

job_description_parser_agent = Agent(
    role="Job Description Parser",
    goal="Analyze job descriptions and extract required skills, qualifications, responsibilities, and experience levels.",
    backstory=(
        "You are an AI-powered job description parser. "
        "Your role is to analyze job descriptions and extract critical information "
        "such as required skills, preferred qualifications, experience levels, and key responsibilities. "
        "You provide a clear and structured breakdown of what the job entails, enabling accurate matching with candidate profiles."
    ),
    allow_delegation=False,
    verbose=True,
    tools=[],
    llm=llm
)

resume_parser_agent = Agent(
    role="Resume Parser",
    goal="Extract detailed information from resumes, including the candidate's name, skills, experience, education, certifications, and contact details (email and phone number).",
    backstory=(
        "You are an AI-powered resume parser with a deep understanding of professional profiles. "
        "Your expertise lies in analyzing resumes and extracting structured data such as the candidate's name, skills, work experience, education, and contact information. "
        "You ensure that no detail is overlooked, providing a comprehensive profile of each candidate for further evaluation."
    ),
    allow_delegation=False,
    verbose=True,
    tools=[],
    llm=llm
)

skill_matcher_agent = Agent(
    role="Skill Matcher",
    goal="Match candidate skills to job requirements and generate a compatibility score.",
    backstory=(
        "You are an AI-powered skill matcher. "
        "Your expertise lies in comparing the skills and qualifications of candidates with the "
        "requirements of job descriptions. You use advanced algorithms to calculate a compatibility score, "
        "ensuring that only the most suitable candidates are shortlisted for further consideration."
    ),
    allow_delegation=False,
    verbose=True,
    tools=[],
    llm=llm
)

candidate_outreach_agent = Agent(
    role="Candidate Outreaching",
    goal="Generate personalized and professional emails for candidate outreach, including the candidate's name, fixed job description details, and assigned interview slot.",
    backstory=(
        "You are an AI-powered email generator. "
        "Your role is to craft personalized and compelling emails for candidates who meet the job requirements. "
        "You highlight their skills and experience, explain why they are a great fit for the role, and include the "
        "candidate's name, fixed job description details, and assigned interview slot that is in future (in March, 2025). "
        "Your emails are professional, engaging, and strictly follow the provided template without adding extra information."
    ),
    allow_delegation=False,
    verbose=True,
    tools=[],
    llm=llm
)

In [ ]:
from crewai import Task

job_description_parsing_task = Task(
    description=(
        "Parse the job description ({job_description}) and extract key requirements, including "
        "required skills, preferred skills, experience levels, and education requirements. "
        "**Do not add any extra information that is not explicitly mentioned in the job-description.** "
        "This task is essential for understanding the job's demands and ensuring accurate matching with candidate profiles."
    ),
    expected_output=(
        "A JSON object containing the job's required skills, preferred skills, experience requirements, "
        "education requirements, and key responsibilities."
    ),
    agent=job_description_parser_agent
)

resume_parsing_task = Task(
    description=(
        "Parse the candidate's resume ({resume_text}) and extract detailed information, including the "
        "candidate's name, skills, experience, education, certifications, and contact details (email and phone number). "
        "**Do not add any extra information that is not explicitly mentioned in the resume.** "
        "This task is critical for building a comprehensive profile of the candidate, which will be used for skill matching and outreach."
    ),
    expected_output=(
        "A JSON object containing the candidate's name, skills, experience, education, certifications, and contact details (email and phone number). "
        "**Do not include any additional fields or information that is not explicitly present in the resume.**"
    ),
    agent=resume_parser_agent
)

skill_matching_task = Task(
    description=(
        "Compare the candidate's skills and qualifications with the job requirements and generate a compatibility score. "
        "**Do not add any extra information that is not explicitly mentioned in the resume.** "
        "This task ensures that only candidates who meet the job's criteria are shortlisted for further consideration."
    ),
    expected_output=(
        "A compatibility score (0 to 1) indicating how well the candidate's skills match the job requirements."
    ),
    agent=skill_matcher_agent
)

candidate_outreach_task = Task(
    description=(
        "Generate a personalized and professional email for the candidate if their compatibility score is above the threshold. "
        "The email should include:\n"
        "1. The candidate's name (extracted from their resume).\n"
        "2. Fixed job description details (required skills, preferred skills, experience levels, education requirements, and key responsibilities).\n"
        "3. The company's details (name, address, contact email, and phone number).\n"
        "4. A call-to-action to confirm the assigned interview slot (which should be in March, 2025).\n"
        "5. A personalized message highlighting the candidate's relevant skills and experience.\n"
        "**Do not add any extra information that is not explicitly provided in the inputs.**"
    ),
    expected_output=(
        "A personalized email draft tailored to the candidate, including:\n"
        "1. The candidate's name from the resume.\n"
        "2. Fixed job description details.\n"
        "3. The company's details.\n"
        "4. The assigned interview slot in March 2025.\n"
        "5. A personalized message and call-to-action.\n"
        "**Do not include any additional information or fields beyond what is explicitly requested.**"
    ),
    tools=[sentiment_analysis_tool],
    agent=candidate_outreach_agent
)

In [8]:
# Function to read content from PDF or DOCX file
def read_file(file_path):
    if file_path.lower().endswith('.pdf'):
        return read_pdf(file_path)
    elif file_path.lower().endswith('.docx'):
        return read_docx(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join(para.text for para in doc.paragraphs)

In [ ]:
from crewai import Crew
from langchain_openai import ChatOpenAI
import json

# Define the crew with agents, tasks, and LLM manager (disable memory to avoid RAG storage issues)
hr_crew = Crew(
    agents=[
        job_description_parser_agent,
        resume_parser_agent,
        skill_matcher_agent,
        candidate_outreach_agent
    ],
    tasks=[
        job_description_parsing_task,
        resume_parsing_task,
        skill_matching_task,
        candidate_outreach_task
    ],
    manager_llm=ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7
    ),
    verbose=True,
    memory=False  # Disable memory to avoid RAG storage errors
)

# Entry point for execution
if __name__ == "__main__":
    # File paths for job description and resumes
    jd_file_path = input("Please enter path to job description: ")
    resume_file_paths = input("Please enter paths to resumes (separated by commas): ").split(',')

    # Read job description
    job_description = read_file(jd_file_path)

    # Process each resume
    selected_candidates = []
    unselected_candidates = []

    for resume_file_path in resume_file_paths:
        resume_text = read_file(resume_file_path.strip())

        # Run the CrewAI pipeline with inputs
        result = hr_crew.kickoff(
            inputs={
                "job_description": job_description,
                "resume_text": resume_text
            }
        )

        # Parse the result as JSON
        if isinstance(result, str):
            try:
                result_dict = json.loads(result)
            except json.JSONDecodeError:
                print(f"Error decoding JSON from result: {result}")
                continue
        elif isinstance(result, dict):
            result_dict = result
        else:
            print(f"Unexpected result type: {type(result)}")
            continue

        # Categorize based on compatibility score
        compatibility_score = result_dict.get("compatibility_score", 0)
        if compatibility_score >= 0.7:
            selected_candidates.append(result_dict)
        else:
            unselected_candidates.append(result_dict)

    # Output selected candidates
    print("\nSelected Candidates")
    for candidate in selected_candidates:
        print(f"Candidate: {candidate.get('candidate_name', 'N/A')}")
        print(f"Compatibility Score: {candidate.get('compatibility_score', 'N/A')}")
        print(f"Email Draft: {candidate.get('email_draft', 'N/A')}\n")

    # Output unselected candidates
    print("\nUnselected Candidates")
    for candidate in unselected_candidates:
        print(f"Candidate: {candidate.get('candidate_name', 'N/A')}")
        print(f"Compatibility Score: {candidate.get('compatibility_score', 'N/A')}\n")